# Concurrent Orchestration with Custom Agent Executors

## Overview

This notebook demonstrates concurrent orchestration using **custom Executor classes** that own their `ChatAgent` instances. Instead of passing agents directly to `ConcurrentBuilder`, we create specialized executor classes that:
1. Instantiate their own `ChatAgent` in `__init__`
2. Define `@handler` methods to process `AgentExecutorRequest` → `AgentExecutorResponse`
3. Enable reuse of the high-level `ConcurrentBuilder` API with custom logic

### Key Benefits:

- **Encapsulation**: Each executor owns its agent and logic
- **Reusability**: Executor classes can be reused across workflows
- **Customization**: Add preprocessing, postprocessing, or state management
- **Type Safety**: Strongly-typed request/response contracts

### Architecture:

```
AgentExecutorRequest (user prompt)
            ↓
    [Internal Dispatcher]
       ↙       ↓        ↘
ResearcherExec  MarketerExec  LegalExec
(custom executors, each with ChatAgent)
       ↘       ↓        ↙
    [Default Aggregator]
            ↓
  list[ChatMessage]
```

## Prerequisites

- Azure OpenAI configured with environment variables
- Azure CLI authentication: Run `az login`
- Agent Framework installed

## Setup and Imports

In [ ]:
import asyncio
from typing import Any

import os
from dotenv import load_dotenv
from agent_framework import (
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatAgent,
    ChatMessage,
    ConcurrentBuilder,
    Executor,
    WorkflowContext,
    handler,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
# Load environment variables from .env file
load_dotenv('../../.env')


## Create Custom Agent Executors

Each executor class follows the same pattern:

### Class Structure:
1. **`agent` attribute**: Stores the ChatAgent instance
2. **`__init__`**: Creates the agent with specific instructions
3. **`@handler` method**: Processes AgentExecutorRequest and emits AgentExecutorResponse

### Handler Contract:
- **Input**: `AgentExecutorRequest` containing messages
- **Output**: `AgentExecutorResponse` with agent response and full conversation
- **Context**: `WorkflowContext[AgentExecutorResponse]` for message routing

In [ ]:
class ResearcherExec(Executor):
    agent: ChatAgent

    def __init__(self, chat_client: AzureOpenAIChatClient, id: str = "researcher"):
        self.agent = chat_client.create_agent(
            instructions=(
                "You're an expert market and product researcher. Given a prompt, provide concise, factual insights,"
                " opportunities, and risks."
            ),
            name=id,
        )
        super().__init__(id=id)

    @handler
    async def run(self, request: AgentExecutorRequest, ctx: WorkflowContext[AgentExecutorResponse]) -> None:
        response = await self.agent.run(request.messages)
        full_conversation = list(request.messages) + list(response.messages)
        await ctx.send_message(AgentExecutorResponse(self.id, response, full_conversation=full_conversation))


class MarketerExec(Executor):
    agent: ChatAgent

    def __init__(self, chat_client: AzureOpenAIChatClient, id: str = "marketer"):
        self.agent = chat_client.create_agent(
            instructions=(
                "You're a creative marketing strategist. Craft compelling value propositions and target messaging"
                " aligned to the prompt."
            ),
            name=id,
        )
        super().__init__(id=id)

    @handler
    async def run(self, request: AgentExecutorRequest, ctx: WorkflowContext[AgentExecutorResponse]) -> None:
        response = await self.agent.run(request.messages)
        full_conversation = list(request.messages) + list(response.messages)
        await ctx.send_message(AgentExecutorResponse(self.id, response, full_conversation=full_conversation))


class LegalExec(Executor):
    agent: ChatAgent

    def __init__(self, chat_client: AzureOpenAIChatClient, id: str = "legal"):
        self.agent = chat_client.create_agent(
            instructions=(
                "You're a cautious legal/compliance reviewer. Highlight constraints, disclaimers, and policy concerns"
                " based on the prompt."
            ),
            name=id,
        )
        super().__init__(id=id)

    @handler
    async def run(self, request: AgentExecutorRequest, ctx: WorkflowContext[AgentExecutorResponse]) -> None:
        response = await self.agent.run(request.messages)
        full_conversation = list(request.messages) + list(response.messages)
        await ctx.send_message(AgentExecutorResponse(self.id, response, full_conversation=full_conversation))

## Build and Execute Workflow

### Key Differences from Direct Agent Usage:

1. **Instantiate executors** instead of passing agents directly
2. **Same ConcurrentBuilder API** - executors are participants
3. **Default aggregator works** - AgentExecutorResponse is compatible

In [ ]:
async def run_concurrent_custom_executors() -> None:
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    chat_client = AzureOpenAIChatClient(
        deployment_name=deployment_name,
        endpoint=endpoint,
        credential=AzureCliCredential()
    )

    researcher = ResearcherExec(chat_client)
    marketer = MarketerExec(chat_client)
    legal = LegalExec(chat_client)

    workflow = ConcurrentBuilder().participants([researcher, marketer, legal]).build()

    events = await workflow.run("We are launching a new budget-friendly electric bike for urban commuters.")
    outputs = events.get_outputs()

    if outputs:
        print("===== Final Aggregated Conversation (messages) =====")
        messages: list[ChatMessage] | Any = outputs[0]  # Get the first (and typically only) output
        for i, msg in enumerate(messages, start=1):
            name = msg.author_name if msg.author_name else "user"
            print(f"{'-' * 60}\n\n{i:02d} [{name}]:\n{msg.text}")

## Run the Workflow

In [ ]:
await run_concurrent_custom_executors()

## Expected Output

Output is identical to concurrent_agents.py, but the implementation uses custom executors.

## Key Takeaways

### 1. Custom Executor Pattern Benefits
- **Encapsulation**: Agent creation and logic bundled in one class
- **Reusability**: Executor instances can be used in multiple workflows
- **Testing**: Easier to unit test executor logic independently
- **State Management**: Can add instance variables for tracking state

### 2. Handler Method Contract
- **Input type**: `AgentExecutorRequest` with `messages` field
- **Output type**: `AgentExecutorResponse` with `executor_id`, `agent_run_response`, and `full_conversation`
- **Context type**: `WorkflowContext[AgentExecutorResponse]` for routing

### 3. Full Conversation Preservation
- Combine `request.messages` + `response.messages`
- Enables downstream executors to see complete context
- Critical for aggregators that need full history

### 4. Flexibility for Customization
You can enhance executors with:
- **Preprocessing**: Modify messages before agent call
- **Postprocessing**: Transform agent responses
- **Caching**: Store and reuse results
- **Logging**: Track agent performance
- **Error handling**: Retry logic or fallbacks

### 5. Compatibility with ConcurrentBuilder
- Custom executors work seamlessly with `.participants([...])`
- Default aggregator handles `AgentExecutorResponse` automatically
- No changes needed to builder code

### 6. Example Enhancements

```python
class CachingResearcher(Executor):
    def __init__(self, chat_client, cache):
        self.agent = chat_client.create_agent(...)
        self.cache = cache
        super().__init__(id="cached_researcher")
    
    @handler
    async def run(self, request, ctx):
        # Check cache first
        cache_key = hash_messages(request.messages)
        if cache_key in self.cache:
            return self.cache[cache_key]
        
        # Call agent if cache miss
        response = await self.agent.run(request.messages)
        self.cache[cache_key] = response
        
        # Send response
        full_conversation = list(request.messages) + list(response.messages)
        await ctx.send_message(
            AgentExecutorResponse(self.id, response, full_conversation=full_conversation)
        )
```

### 7. When to Use Custom Executors vs. Direct Agents

| Use Custom Executors When: | Use Direct Agents When: |
|----------------------------|-------------------------|
| Need preprocessing/postprocessing | Simple, straightforward agent calls |
| Want state management | Stateless execution |
| Complex error handling needed | Default error handling sufficient |
| Reusing executor across workflows | One-off workflow usage |
| Need custom logging/monitoring | Standard logging is enough |

### 8. Production Considerations
- Add proper error handling in handler methods
- Consider timeout handling for slow agents
- Log executor-specific metrics (response time, token usage)
- Implement retry logic for transient failures
- Add circuit breakers for failing agents